In [7]:
import sqc
import numpy as np
from exercises import cqft

# Addition through Fourier transform with control bit

In [8]:
def CRz(i,t,phi,o):
    return o.Rz(t,phi/2.).CNOT(i,t).Rz(t,-phi/2.).Rz(i,phi/2.).CNOT(i,t)

def add(c,y,op,xbits):
    op=cqft(c,op,mask=xbits)
    N=len(xbits)
    for i in range(N):
        op=CRz(c,xbits[i],2.*np.pi*y/2.**(N-i),op)
    op=cqft(c,op,inverse=True,mask=xbits)
    return op

st=sqc.state(6,basis=["|%d>|%d>" % (i % 2**5,i//2**5) for i in range(2**6)])

xbits=[0,1,2,3,4]
cbit=5

st=sqc.operator(6).H(cbit)*st
print("Initial = 0\n",st)

st=add(cbit,3,sqc.operator(6),xbits=xbits)*st
print("Initial + 3\n",st)
st=add(cbit,7,sqc.operator(6),xbits=xbits)*st
print("Initial + 3 + 7\n",st)

Initial = 0
    0.707107 * |0>|0>
 + 0.707107 * |0>|1>
Initial + 3
    0.707107 * |0>|0>
 + 0.707107 * |3>|1>
Initial + 3 + 7
    0.707107 * |0>|0>
 + 0.707107 * |10>|1>


# Precompute ci

In [9]:
def bits(y,N):
    return [ (y//2**j) % 2 for j in range(N) ]

def ci(x,n,N):
    b=bits(x,N)
    return [ sum([ b[j]*2**(i+j) for j in range(N) ]) % n for i in range(N) ]



In [5]:
print(ci(5,21,5))

[5, 10, 20, 19, 17]


# Multiplication by constant (without mod n)

In [17]:
def mult(x,op,xbits,tbits):
    Nbits=len(xbits)
    c=ci(x,2**Nbits,Nbits)
    for i in range(Nbits):
        op=add(xbits[i],c[i],op,tbits)
    return op

st=sqc.state(6,basis=["|%d>|%d>" % (i % 2**3,i//2**3) for i in range(2**6)])

xbits=[0,1,2]
tbits=[3,4,5]

st=sqc.operator(6).X(1)*st
print("Initial state\n",st)

st=mult(3,sqc.operator(6),xbits=xbits,tbits=tbits)*st
print("Mult by 3\n",st)

Initial state
    1 * |2>|0>
Mult by 3
    1 * |2>|6>
